In [1]:
import requests

In [64]:
r = requests.get('http://www.patentsview.org/api/patents/query?q={"patent_type":"design"}&f=["patent_number","patent_title"]')

In [65]:
r.json()

{'count': 25,
 'patents': [{'patent_number': '4198236',
   'patent_title': 'Method for preparation of lithographic printing plate having addition polymerized areas and binder areas'},
  {'patent_number': '4366172',
   'patent_title': '4-Amino-cyclohexanols, their pharmaceutical compositions and methods of use'},
  {'patent_number': '4371816',
   'patent_title': 'Control circuit for an ultrasonic dental scaler'},
  {'patent_number': '4419147',
   'patent_title': 'Phosphating process of metal surface'},
  {'patent_number': '4451875',
   'patent_title': 'Poster panel lighting fixture'},
  {'patent_number': '4461003',
   'patent_title': 'Circuit arrangement for preventing a microcomputer from malfunctioning'},
  {'patent_number': '4486659', 'patent_title': 'Emisson-electron microscope'},
  {'patent_number': '4544582',
   'patent_title': 'Primer compositions for improving adhesion of abrasion resistant silicone coatings to plastic surfaces'},
  {'patent_number': '4547613',
   'patent_title'